# DATABASE APPLICATION & DATA SCIENCE

Fazer um algoritmo em Python que leia um arquivo .csv relacionado ao tema de compra de passagens de ônibus e armazenar em uma coleção. Varrer a coleção preenchida e carregar uma tabela Oracle.

## Estrutura do arquivo

O arquivo contém registros no formato para detalhar o retorno de somente uma linha. Este retorno está representando uma transação de compra de passagens de ônibus. A seguir, detalhamos cada um dos campos presentes em um exemplo de registro.

In [0]:
{
  "nk_ota_localizer_id": "3H0YV76C74V8",
  "fk_contact": "16SHI8GNRK",
  "date_purchase": "2020-11-09",
  "time_purchase": "17:49:43",
  "place_origin_departure": "Natal",
  "place_destination_departure": "São Luís",
  "place_origin_return": "São Luís",
  "place_destination_return": "Natal",
  "fk_departure_ota_bus_company": "Rod Rotas",
  "fk_return_ota_bus_company": "Expresso Itamarati",
  "gmv_success": 494.12,
  "total_tickets_quantity_success": 5
}

## Instalação do Pacote OracleDB

Instalação da biblioteca `oracledb` para permitir a conexão com bancos de dados Oracle, pois não há isso de forma nativa no Databricks.

O comando `%pip install oracledb` instala o pacote oracledb via pip no ambiente do Databricks. O pacote oracledb é utilizado para estabelecer conexões com bancos de dados Oracle e realizar operações como consultas, inserções e atualizações.

Enquanto o comando `dbutils.library.restartPython()` reinicia o kernel do Python no Databricks, recarregando as bibliotecas e dependências instaladas recentemente. O uso do restartPython() é frequentemente necessário após a instalação de novas bibliotecas, para garantir que as alterações no ambiente sejam aplicadas corretamente.

In [0]:
%pip install oracledb
dbutils.library.restartPython()

## Execução do Algoritmo

#### Leitura de Dados e Conversão para Pandas

**Descrição**: Converte dados do Databricks para um DataFrame do Pandas e os organiza em um formato de lista de dicionários.

- O código extrai os 1000 primeiros registros da tabela bus.bronze.ota_bus_ticket_sales no Databricks usando o Spark, converte esses dados para um DataFrame do Pandas e, em seguida, os transforma em uma lista de dicionários, que será utilizada para inserir dados no banco de dados Oracle.

#### Conexão com Banco de Dados Oracle

**Descrição**: Estabelece uma conexão com o banco de dados Oracle usando as credenciais.

- O código tenta se conectar ao banco de dados Oracle utilizando as credenciais fornecidas (usuário, senha e DSN). Se a conexão for bem-sucedida, imprime uma mensagem de sucesso, caso contrário, captura o erro e interrompe a execução do programa.

#### Inserção de Dados no Banco de Dados Oracle

**Descrição**: Insere os dados coletados no banco de dados Oracle utilizando uma consulta SQL.

- O código prepara e executa a inserção dos dados coletados na coleção no banco de dados Oracle. A consulta SQL é parametrizada para prevenir SQL Injection, e o processo de inserção é executado para cada linha de dados. Após a inserção, as mudanças são confirmadas com o **commit()**, e a conexão é fechada.

In [0]:
import pandas as pd
import oracledb

df_ota_bus_ticket_sales = spark.table("bus.bronze.ota_bus_ticket_sales").limit(1000)

df_pandas = df_ota_bus_ticket_sales.toPandas()
colecao = df_pandas.to_dict(orient="records")

print("Prévia da coleção extraída do nosso Dataframe (Camada Bronze):")
for item in colecao[:3]:
    print(item)

try:
    conn = oracledb.connect(
        user="RM562679",
        password="260701",
        dsn="oracle.fiap.com.br:1521/orcl"
    )
    print("Conexão bem sucedida!")
except Exception as e:
    print(f"Erro de conexão: {e}")
    exit()

try:
    cur = conn.cursor()
    
    for linha in colecao:
        consulta = """
        INSERT INTO ota_bus_ticket_sales (
            nk_ota_localizer_id,
            fk_contact,
            date_purchase,
            time_purchase,
            place_origin_departure,
            place_destination_departure,
            place_origin_return,
            place_destination_return,
            fk_departure_ota_bus_company,
            fk_return_ota_bus_company,
            gmv_success,
            total_tickets_quantity_success
        ) VALUES (
            :nk_ota_localizer_id,
            :fk_contact,
            :date_purchase,
            :time_purchase,
            :place_origin_departure,
            :place_destination_departure,
            :place_origin_return,
            :place_destination_return,
            :fk_departure_ota_bus_company,
            :fk_return_ota_bus_company,
            :gmv_success,
            :total_tickets_quantity_success
        )
        """
        
        cur.execute(consulta, linha)
    
    conn.commit()
    print("Dados inseridos com sucesso no Oracle!")
    
except Exception as e:
    print(f"Erro ao inserir dados: {e}")

finally:
    cur.close()
    conn.close()

## Criação da tabela no Banco OracleDB

Este comando SQL cria a tabela **ota_bus_ticket_sales** no banco de dados Oracle. Cada coluna da tabela corresponde a um campo da coleção de dados de vendas de passagens de ônibus. A coluna **nk_ota_localizer_id** é definida como a chave primária, garantindo que cada registro tenha um identificador único.

In [0]:
%sql
CREATE TABLE ota_bus_ticket_sales (
    nk_ota_localizer_id        VARCHAR2(20) PRIMARY KEY,
    fk_contact                 VARCHAR2(20),
    date_purchase              DATE,
    time_purchase              VARCHAR2(10),
    place_origin_departure     VARCHAR2(100),
    place_destination_departure VARCHAR2(100),
    place_origin_return        VARCHAR2(100),
    place_destination_return   VARCHAR2(100),
    fk_departure_ota_bus_company VARCHAR2(100),
    fk_return_ota_bus_company  VARCHAR2(100),
    gmv_success                NUMBER(10,2),
    total_tickets_quantity_success NUMBER(5)
);